In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
import matplotlib.pyplot as plt
from glob import glob
from utils.geotif_io import readTiff, writeTiff
import xdem
from utils.lay_stack import lay_stack
import xarray as xr


In [4]:
path_1 = 'data/icesat-1/tiles-dif-srtm/tile_27_96.h5'
path_2 = 'data/icesat-1/tiles-dif-srtm/tile_27_97.h5'
f1_read = h5py.File(path_1)
# f2_read = h5py.File(path_2)
!h5ls $path_1


h                        Dataset {68}
h_dif                    Dataset {68}
h_srtm                   Dataset {68}
lat                      Dataset {68}
lon                      Dataset {68}
t_dyr                    Dataset {68}
type_fp                  Dataset {68}


In [5]:
ids = np.where(f1_read['h_dif'][:]>-20)[0]
ids_ = np.concatenate((ids,ids), axis=0) 
ids_ 


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

In [6]:
!h5ls $path




usage: h5ls [OPTIONS] file[/OBJECT] [file[/[OBJECT]...]
  OPTIONS
   -h, -?, --help  Print a usage message and exit
   -a, --address   Print raw data address.  If dataset is contiguous, address
                   is offset in file of beginning of raw data. If chunked,
                   returned list of addresses indicates offset of each chunk.
                   Must be used with -v, --verbose option.
                   Provides no information for non-dataset objects.
   -d, --data      Print the values of datasets
   --enable-error-stack
                   Prints messages from the HDF5 error stack as they occur.
   --follow-symlinks
                   Follow symbolic links (soft links and external links)
                   to display target object information.
                   Without this option, h5ls identifies a symbolic link
                   as a soft link or external link and prints the value
                   assigned to the symbolic link; it does not provide any
         

In [7]:
path = 'data/icesat-1/stat_dif_isat1.nc'



In [8]:
stat_isat1_xr = xr.open_dataset(path)
stat_isat1_xr


<xarray.Dataset>
Dimensions:                   (tiles_id: 28, years: 7)
Coordinates:
  * tiles_id                  (tiles_id) object 'tile_27_96' ... 'tile_31_95'
  * years                     (years) object '2003' '2004' ... '2008' '2009'
Data variables:
    area_glacier_tiles        (tiles_id) float64 ...
    mean_glacier_tiles_years  (tiles_id, years) float64 ...
    std_glacier_tiles_years   (tiles_id, years) float64 ...
    mean_stable_tiles_years   (tiles_id, years) float64 ...
    std_stable_tiles_years    (tiles_id, years) float64 ...

In [14]:
def outlier_remove_sigma(data, coef_sigma):
    if data.shape[0] != 0:
        mean_dif_year = np.mean(data)
        sigma_dif_year = np.std(data)
        # print(sigma_dif_year)
        thre_max, thre_min = mean_dif_year+coef_sigma*sigma_dif_year, mean_dif_year-coef_sigma*sigma_dif_year
        # print(thre_max, thre_min)
        ids_filter = np.where((data > thre_min) & (data < thre_max))
        print(ids_filter)
        data_filter = data[ids_filter]
    else: data_filter = np.array([])
    return data_filter



In [19]:
data = np.array([1,2,1,2,5,3,6,7,3,4,4,20,50])
# data = np.array([])
a = outlier_remove_sigma(data=data, coef_sigma=1)



(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),)


In [22]:
ids_filter = np.where((data < 20) & (data > 2))
ids_filter 
data[ids_filter]


array([5, 3, 6, 7, 3, 4, 4])